# TCX Bike Race Weight Power Analysis

This notebook analyzes the impact of an additional 1kg on normalized power output during bike races by modeling energy requirements from velocity changes and elevation gain. We use a worst-case scenario approach that ignores benefits from downhill sections.

## Overview
- Parses TCX files from bike race recordings
- Calculates kinetic energy changes from velocity variations
- Calculates gravitational potential energy changes from elevation gain
- Estimates the extra power in Watts required for 1kg additional mass
- Computes normalized power impact

## Section 1: Import Required Libraries

In [ ]:
import xml.etree.ElementTree as ET
from pathlib import Path
from dataclasses import dataclass
from typing import List, Dict
import statistics
import json

# Visualization libraries
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import os

# Get all TCX files in the workspace
tcx_files = list(Path('/workspaces/np_weight_analysis').glob('*.tcx'))
print(f"Found {len(tcx_files)} TCX files:")
for f in tcx_files:
    print(f"  - {f.name}")

## Section 2: Load and Parse TCX File

In [ ]:
# Define physical constants
G = 9.81  # gravitational acceleration (m/s²)

@dataclass
class TrackPoint:
    """A single trackpoint from the TCX file."""
    time: str
    latitude: float
    longitude: float
    elevation: float
    distance: float
    speed: float
    cadence: int = None
    heart_rate: int = None

class TCXAnalyzer:
    """Parses and analyzes TCX bike race files."""
    
    def __init__(self, tcx_file_path: str):
        """Initialize the analyzer with a TCX file."""
        self.file_path = tcx_file_path
        self.trackpoints = []
        self.parse_tcx()
        
    def parse_tcx(self):
        """Parse TCX file and extract trackpoints."""
        tree = ET.parse(self.file_path)
        root = tree.getroot()
        
        # Define namespaces
        ns = {
            'ns': 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2',
            'ns3': 'http://www.garmin.com/xmlschemas/ActivityExtension/v2'
        }
        
        # Find all trackpoints
        for tp in root.findall('.//ns:Trackpoint', ns):
            try:
                time_elem = tp.find('ns:Time', ns)
                position = tp.find('ns:Position', ns)
                elev_elem = tp.find('ns:AltitudeMeters', ns)
                dist_elem = tp.find('ns:DistanceMeters', ns)
                cadence_elem = tp.find('ns:Cadence', ns)
                hr_elem = tp.find('ns:HeartRateBpm/ns:Value', ns)
                
                # Get speed from extensions
                speed = 0
                tpx = tp.find('.//ns3:TPX', ns)
                if tpx is not None:
                    speed_elem = tpx.find('ns3:Speed', ns)
                    if speed_elem is not None and speed_elem.text:
                        speed = float(speed_elem.text)
                
                if (time_elem is not None and position is not None and 
                    elev_elem is not None and dist_elem is not None):
                    
                    lat = float(position.find('ns:LatitudeDegrees', ns).text)
                    lon = float(position.find('ns:LongitudeDegrees', ns).text)
                    
                    tp_obj = TrackPoint(
                        time=time_elem.text,
                        latitude=lat,
                        longitude=lon,
                        elevation=float(elev_elem.text),
                        distance=float(dist_elem.text),
                        speed=speed,
                        cadence=int(cadence_elem.text) if cadence_elem is not None and cadence_elem.text else None,
                        heart_rate=int(hr_elem.text) if hr_elem is not None and hr_elem.text else None
                    )
                    self.trackpoints.append(tp_obj)
            except (ValueError, AttributeError) as e:
                continue

# Test parsing with the first file
if tcx_files:
    test_file = tcx_files[0]
    print(f"\nParsing: {test_file.name}")
    analyzer = TCXAnalyzer(str(test_file))
    print(f"Successfully parsed {len(analyzer.trackpoints)} trackpoints")
    if analyzer.trackpoints:
        tp = analyzer.trackpoints[0]
        print(f"First trackpoint: {tp.time}, Speed: {tp.speed:.2f} m/s, Elevation: {tp.elevation:.1f}m")

## Section 3: Calculate Energy Requirements and Power Impact

Now we'll analyze the power impact of 1kg extra weight by calculating:
- **Kinetic Energy Change**: $\Delta KE = \frac{1}{2}m(\Delta v)^2$
- **Potential Energy Change**: $\Delta PE = mg\Delta h$ (for uphill only in worst-case)
- **Extra Power Cost**: $P = \frac{\Delta E}{\Delta t}$

In [ ]:
def analyze_power_impact(analyzer: TCXAnalyzer, rider_mass: float = 75.0, extra_weight: float = 1.0):
    """
    Calculate the power impact of extra weight due to kinetic and gravitational PE changes.
    
    Arguments:
        analyzer: TCXAnalyzer instance with parsed trackpoints
        rider_mass: rider + bike mass in kg (default 75 kg)
        extra_weight: additional weight in kg (default 1 kg)
    
    Returns:
        Dictionary containing analysis results
    """
    if len(analyzer.trackpoints) < 2:
        return None
    
    total_mass = rider_mass + extra_weight
    base_mass = rider_mass
    
    # Initialize tracking variables
    kinetic_power = []  # watts needed for kinetic energy (base + extra)
    potential_power = []  # watts needed for gravitational PE
    extra_kinetic_power = []  # extra watts from kinetic energy with extra weight
    extra_potential_power = []  # extra watts from potential energy with extra weight
    total_extra_power = []  # total extra watts needed
    timestamps = []
    velocities = []
    elevation_deltas = []
    
    prev_tp = analyzer.trackpoints[0]
    total_elev_gain = 0
    total_distance = 0
    
    for i, tp in enumerate(analyzer.trackpoints[1:], start=1):
        # Time difference (1 second for TCX files)
        dt = 1.0
        
        # Current velocity
        v_current = tp.speed  # m/s
        v_prev = prev_tp.speed  # m/s
        
        # Elevation change
        elev_delta = tp.elevation - prev_tp.elevation
        if elev_delta > 0:
            total_elev_gain += elev_delta
        
        # Kinetic energy change: ½m(v² - v₀²)
        # Power = ΔKE / Δt
        ke_change_base = 0.5 * base_mass * (v_current**2 - v_prev**2) if dt > 0 else 0
        ke_change_extra = 0.5 * total_mass * (v_current**2 - v_prev**2) if dt > 0 else 0
        
        # Power from kinetic energy for extra weight
        kinetic_pw_base = ke_change_base / dt if dt > 0 else 0
        kinetic_pw_extra = ke_change_extra / dt if dt > 0 else 0
        kinetic_pw_delta = kinetic_pw_extra - kinetic_pw_base
        
        # Gravitational PE change: mg*Δh (only uphill in worst case)
        potential_pw_delta = 0
        if elev_delta > 0:
            pe_change_base = base_mass * G * elev_delta
            pe_change_extra = total_mass * G * elev_delta
            pe_pw_base = pe_change_base / dt if dt > 0 else 0
            pe_pw_extra = pe_change_extra / dt if dt > 0 else 0
            potential_pw_delta = pe_pw_extra - pe_pw_base
        
        # Total extra power needed (worst case: no benefits from downhill)
        total_delta = max(0, kinetic_pw_delta + potential_pw_delta)
        
        extra_kinetic_power.append(kinetic_pw_delta)
        extra_potential_power.append(potential_pw_delta)
        total_extra_power.append(total_delta)
        timestamps.append(tp.time)
        velocities.append(v_current)
        elevation_deltas.append(elev_delta)
        
        prev_tp = tp
    
    # Calculate statistics
    # Normalized Power (similar to TrainingPeaks algorithm, simplified)
    # NP = (avg(power^4))^(1/4)
    if len(total_extra_power) > 0:
        power_4 = [p**4 for p in total_extra_power]
        mean_power_4 = sum(power_4) / len(power_4)
        np_extra = mean_power_4**0.25 if mean_power_4 > 0 else 0
    else:
        np_extra = 0
    
    # Average power
    avg_power_extra = sum(total_extra_power) / len(total_extra_power) if len(total_extra_power) > 0 else 0
    max_power_extra = max(total_extra_power) if len(total_extra_power) > 0 else 0
    
    # Total energy (Joules)
    total_energy = sum(total_extra_power) if len(total_extra_power) > 0 else 0
    
    # Duration
    duration_seconds = len(analyzer.trackpoints) - 1
    
    return {
        'file_name': Path(analyzer.file_path).name,
        'duration_seconds': duration_seconds,
        'distance_km': analyzer.trackpoints[-1].distance / 1000 if analyzer.trackpoints else 0,
        'elevation_gain_m': total_elev_gain,
        'max_speed_kmh': max(velocities) * 3.6 if velocities else 0,
        'avg_speed_kmh': (sum(velocities) / len(velocities) * 3.6) if velocities else 0,
        'powers': {
            'normalized_power_watts': np_extra,
            'average_power_watts': avg_power_extra,
            'max_power_watts': max_power_extra,
            'total_energy_joules': total_energy,
            'total_energy_kcal': total_energy / 4184
        },
        'rider_mass_kg': rider_mass,
        'extra_weight_kg': extra_weight,
        'power_data': {
            'timestamps': timestamps,
            'extra_total_power': total_extra_power,
            'extra_kinetic_power': extra_kinetic_power,
            'extra_potential_power': extra_potential_power,
            'velocities': velocities,
            'elevation_deltas': elevation_deltas
        }
    }

# Analyze all TCX files
results = []
for tcx_file in tcx_files:
    try:
        analyzer = TCXAnalyzer(str(tcx_file))
        result = analyze_power_impact(analyzer)
        if result:
            results.append(result)
    except Exception as e:
        print(f"Error processing {tcx_file.name}: {e}")

print(f"\nSuccessfully analyzed {len(results)} files")
for r in results:
    print(f"\n{r['file_name']}:")
    print(f"  Duration: {r['duration_seconds']/60:.1f} min")
    print(f"  Distance: {r['distance_km']:.1f} km")
    print(f"  Elevation: {r['elevation_gain_m']:.0f}m")
    print(f"  Avg speed: {r['avg_speed_kmh']:.1f} km/h")
    print(f"  Extra 1kg NP: {r['powers']['normalized_power_watts']:.1f}W")
    print(f"  Extra 1kg Avg: {r['powers']['average_power_watts']:.1f}W")
    print(f"  Extra 1kg Max: {r['powers']['max_power_watts']:.1f}W")
    print(f"  Total Energy: {r['powers']['total_energy_kcal']:.1f} kcal")

## Section 4: Visualize Results

In [ ]:
# Create visualizations for each race
for result in results[:3]:  # Show first 3 races
    fig, axes = plt.subplots(3, 1, figsize=(14, 10))
    fig.suptitle(f"Weight Impact Analysis: {result['file_name']}", fontsize=14, fontweight='bold')
    
    power_data = result['power_data']
    time_seconds = list(range(len(power_data['extra_total_power'])))
    
    # Plot 1: Extra power cost over time
    ax = axes[0]
    ax.fill_between(time_seconds, 0, power_data['extra_total_power'], 
                     label='Total Extra Power', alpha=0.7, color='red')
    ax.axhline(y=result['powers']['average_power_watts'], color='darkred', 
               linestyle='--', label=f"Avg: {result['powers']['average_power_watts']:.1f}W")
    ax.set_ylabel('Extra Power (Watts)')
    ax.set_title('Extra Power Cost for 1kg')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 2: Velocity profile
    ax = axes[1]
    ax.plot(time_seconds, [v*3.6 for v in power_data['velocities']], 
            color='blue', linewidth=1.5, label='Speed')
    ax.fill_between(time_seconds, 0, [v*3.6 for v in power_data['velocities']], 
                     alpha=0.2, color='blue')
    ax.set_ylabel('Speed (km/h)')
    ax.set_title('Velocity Profile')
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    # Plot 3: Elevation gain
    ax = axes[2]
    cumulative_elev = []
    total_gain = 0
    for elev_delta in power_data['elevation_deltas']:
        if elev_delta > 0:
            total_gain += elev_delta
        cumulative_elev.append(total_gain)
    
    ax.fill_between(time_seconds, 0, cumulative_elev, 
                     label='Cumulative Elevation Gain', alpha=0.5, color='green')
    ax.set_xlabel('Time (seconds)')
    ax.set_ylabel('Elevation Gain (m)')
    ax.set_title('Elevation Profile (Uphill Only)')
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    plt.tight_layout()
    plt.show()

# Summary statistics
print("\n" + "="*80)
print("RACE COMPARISON SUMMARY")
print("="*80)

for r in results:
    print(f"\n{r['file_name']}")
    print(f"{'Duration':<20} {r['duration_seconds']/60:>8.1f} min")
    print(f"{'Distance':<20} {r['distance_km']:>8.1f} km")
    print(f"{'Elevation Gain':<20} {r['elevation_gain_m']:>8.0f} m")
    print(f"{'Avg/Max Speed':<20} {r['avg_speed_kmh']:>6.1f}/{r['max_speed_kmh']:>5.1f} km/h")
    print(f"\n{'1kg Extra Weight Impact:':<20}")
    print(f"{'  Normalized Power':<20} {r['powers']['normalized_power_watts']:>8.1f} W")
    print(f"{'  Average Power':<20} {r['powers']['average_power_watts']:>8.1f} W")
    print(f"{'  Max Power':<20} {r['powers']['max_power_watts']:>8.1f} W")
    print(f"{'  Total Energy':<20} {r['powers']['total_energy_kcal']:>8.1f} kcal")

## Section 5: Detailed Analysis and Insights

In [ ]:

# Detailed insights
print("="*80)
print("DETAILED INSIGHTS")
print("="*80)

print("\nAPPROACH EXPLANATION:")
print("-" * 80)
print("""
This analysis calculates the extra power needed for 1kg additional weight using:

1. KINETIC ENERGY TERM:
   - When velocity changes, energy is needed: ΔKE = ½m(v₂² - v₁²)
   - Extra power from 1kg = ½ × 1kg × (v₂² - v₁²) / Δt
   - This captures acceleration and deceleration costs

2. GRAVITATIONAL POTENTIAL ENERGY TERM:
   - Climbing: ΔPE = mg × Δh (only uphill in worst-case)
   - Extra power from 1kg = 1kg × g × Δh / Δt
   - Downhill benefit is ignored for worst-case analysis

3. NORMALIZED POWER:
   - Standard cycling metric: NP = (mean(power⁴))^(1/4)
   - Better represents effort on variable power rides than simple average
   - Penalizes high peaks and smooth power more appropriately

IMPORTANT LIMITATIONS OF WORST-CASE ANALYSIS:
   - Ignores momentum benefits on descents (worst-case assumption)
   - Ignores aerodynamic benefits of 1kg weight reduction
   - Does not account for rolling resistance (which increases slightly)
   - Real impact depends on speed (higher speeds = more aero loss matters)
""")

print("\nKEY FINDINGS:")
print("-" * 80)

if results:
    avg_np = statistics.mean([r['powers']['normalized_power_watts'] for r in results])
    max_np = max([r['powers']['normalized_power_watts'] for r in results])
    min_np = min([r['powers']['normalized_power_watts'] for r in results])
    
    total_kcal = sum([r['powers']['total_energy_kcal'] for r in results])
    total_hours = sum([r['duration_seconds'] for r in results]) / 3600
    
    print(f"\nAcross all {len(results)} races:")
    print(f"  Average 1kg NP cost: {avg_np:.1f}W")
    print(f"  Range: {min_np:.1f}W - {max_np:.1f}W")
    print(f"  Total extra energy: {total_kcal:.0f} kcal")
    print(f"  Total time: {total_hours:.1f} hours")
    
    # Find which race has most/least elevation
    by_elev = sorted(results, key=lambda r: r['elevation_gain_m'], reverse=True)
    print(f"\n  Most elevation: {by_elev[0]['file_name']} ({by_elev[0]['elevation_gain_m']:.0f}m)")
    print(f"  Least elevation: {by_elev[-1]['file_name']} ({by_elev[-1]['elevation_gain_m']:.0f}m)")
    
    # Find which race has highest intensity
    by_power = sorted(results, key=lambda r: r['powers']['normalized_power_watts'], reverse=True)
    print(f"\n  Highest 1kg power cost: {by_power[0]['file_name']} ({by_power[0]['powers']['normalized_power_watts']:.1f}W)")
    print(f"  Lowest 1kg power cost: {by_power[-1]['file_name']} ({by_power[-1]['powers']['normalized_power_watts']:.1f}W)")

# Save results to JSON
output_file = Path('/workspaces/np_weight_analysis/weight_analysis_results.json')
with open(output_file, 'w') as f:
    # Convert to JSON-serializable format
    json_results = []
    for r in results:
        json_results.append({
            'file_name': r['file_name'],
            'duration_seconds': r['duration_seconds'],
            'distance_km': r['distance_km'],
            'elevation_gain_m': r['elevation_gain_m'],
            'avg_speed_kmh': r['avg_speed_kmh'],
            'max_speed_kmh': r['max_speed_kmh'],
            'normalized_power_watts': r['powers']['normalized_power_watts'],
            'average_power_watts': r['powers']['average_power_watts'],
            'max_power_watts': r['powers']['max_power_watts'],
            'total_energy_kcal': r['powers']['total_energy_kcal'],
        })
    json.dump(json_results, f, indent=2)
    
print(f"\nResults saved to: weight_analysis_results.json")